In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


In [4]:
model.children

<bound method Module.modules of TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [5]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
    
model.pool.register_forward_hook(layer1_hook)

In [6]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [7]:
outputs

[tensor([[[[1.2208e+00, 4.1855e-01, 3.5690e-01,  ..., 5.2678e-01,
            6.8515e-01, 6.1925e-01],
           [6.6051e-01, 1.2432e+00, 3.0593e-01,  ..., 1.2898e+00,
            8.7232e-01, 8.6465e-01],
           [1.2476e+00, 5.3795e-01, 5.9966e-01,  ..., 5.3252e-01,
            5.0231e-01, 1.0131e+00],
           ...,
           [3.7171e-01, 4.5913e-01, 1.1293e+00,  ..., 4.6897e-01,
            4.5439e-01, 7.2753e-01],
           [5.0514e-01, 9.8019e-01, 1.3255e+00,  ..., 6.6641e-01,
            6.5042e-01, 6.9652e-01],
           [6.6299e-01, 1.2846e-01, 8.6551e-01,  ..., 7.1681e-01,
            2.8841e-01, 1.3361e+00]],
 
          [[3.6869e-01, 1.1723e+00, 1.2000e+00,  ..., 6.0000e-01,
            2.0813e+00, 7.2818e-01],
           [0.0000e+00, 7.3088e-01, 8.1768e-01,  ..., 7.9706e-01,
            8.0467e-01, 6.0701e-01],
           [6.5948e-01, 4.1971e-01, 0.0000e+00,  ..., 7.4460e-01,
            1.5810e-01, 6.4020e-01],
           ...,
           [5.8780e-01, 3.6043e-01, 6.

In [8]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [9]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [10]:
for name, parameters in model.named_parameters():
    print(name)
        

conv1.weight
conv1.bias
conv2.weight
conv2.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
fc3.weight
fc3.bias


In [11]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


